In [1]:
import pymssql
import re
from collections import OrderedDict
from pandas import Series,DataFrame
from tqdm import tqdm_notebook
import pandas as pd
import copy

import numpy as np

import pickle

import tensorflow as tf

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,Convolution2D,Activation,MaxPooling2D,Concatenate
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.models import load_model
from sklearn.model_selection import train_test_split

import os

Using TensorFlow backend.


In [2]:
with open('shoda4000.pickle','rb') as fr: #피클 파일 불러와서 합치기
    result3 = pickle.load(fr)

In [3]:
result3.LABEL.value_counts()

0    3836
2    3560
6    3114
1    3076
4    2991
3    2518
7    2455
5    2396
Name: LABEL, dtype: int64

In [4]:
#다른 크기 이미지 제거후 인덱스 섞어주기
for index in result3.index:
    if result3.loc[index]['pl_imgurl'].shape != (28,28,3): 
        result3.drop(index, inplace = True)
result3 = pd.concat([result3],ignore_index=True) 
result3 = result3.reindex(np.random.permutation(result3.index)) 

In [5]:
#원핫벡터 정답 label 만들기
from keras.utils.np_utils import to_categorical
labels = to_categorical(result3['LABEL'], num_classes =8) 
print(labels[:10])

[[0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]]


In [12]:
#트레이닝 테스트 셋 분할
data_train_image,data_test_image,labels_train_image,labels_test_image = train_test_split(np.array(result3['pl_imgurl'].tolist()),labels, test_size = 0.25, random_state=42)

In [13]:
#cnn모델 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()
keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')


model = Sequential()
model.add(Convolution2D(16, 3, 3, border_mode='same', activation='relu', #3x3필터 16개를 적용, same:입력이미지와 출력 이미지크기같음
                        input_shape=data_train_image.shape[1:]))

model.add(MaxPooling2D(pool_size=(2, 2))) #2,2 중 가장 높은값을 가져옴 - 사소한 특징 무시
model.add(Dropout(0.25))
  
model.add(Convolution2D(64, 3, 3,  activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Convolution2D(64, 3, 3))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
  
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(8,activation = 'softmax'))
  
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(data_train_image, labels_train_image, batch_size=200,validation_split = 0.2, nb_epoch=100,callbacks=[early_stopping])

# model.save('Gersang.h5')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", input_shape=(28, 28, 3..., padding="same")`
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 14341 samples, validate on 3586 samples
Epoch 1/100
14341/14341 [==============================] - ETA: 1:56 - loss: 2.1052 - acc: 0.120 - ETA: 1:02 - loss: 2.0960 - acc: 0.115 - ETA: 43s - loss: 2.0835 - acc: 0.131 - ETA: 34s - loss: 2.0783 - acc: 0.14 - ETA: 28s - loss: 2.0762 - acc: 0.13 - ETA: 25s - loss: 2.0754 - acc: 0.13 - ETA: 22s - loss: 2.0793 - acc: 0.13 - ETA: 20s - loss: 2.0778 - acc: 0.13 - ETA: 18s - loss: 2.0785 - acc: 0.13 - ETA: 17s - loss: 2.0752 - acc: 0.14 - ETA: 16s - loss: 2.0733 - acc: 0.14 - ETA: 15s - loss: 2.0717 - acc: 0.14 - ETA: 14s - loss: 2.0707 - acc: 0.14 - ETA: 13s - loss: 2.0717 - acc: 0.14 - ETA: 13s - loss: 2.0708 - acc: 0.14 - ETA: 12s - loss: 2.0703 - acc: 0.14 - ETA: 12s - loss: 2.0698 - acc: 0.14 - ETA: 11s - loss: 2.0692 - acc: 0.14 - ETA: 11s - loss: 2.0682 - acc: 0.14 - ETA: 10s - loss: 2.0670 - acc: 0.15 - ETA: 10s - loss: 2.0661 - acc: 0.15 - ETA: 10s - loss: 2.0649 - acc: 0.15 - ETA: 9s - loss: 2.0647 - acc: 0.1515 - ETA: 9s - lo

14341/14341 [==============================] - ETA: 9s - loss: 1.7615 - acc: 0.320 - ETA: 9s - loss: 1.7556 - acc: 0.347 - ETA: 9s - loss: 1.7669 - acc: 0.326 - ETA: 8s - loss: 1.7772 - acc: 0.331 - ETA: 8s - loss: 1.7870 - acc: 0.318 - ETA: 8s - loss: 1.7744 - acc: 0.320 - ETA: 8s - loss: 1.7723 - acc: 0.325 - ETA: 8s - loss: 1.7691 - acc: 0.324 - ETA: 8s - loss: 1.7758 - acc: 0.320 - ETA: 8s - loss: 1.7724 - acc: 0.323 - ETA: 7s - loss: 1.7717 - acc: 0.327 - ETA: 7s - loss: 1.7629 - acc: 0.334 - ETA: 7s - loss: 1.7564 - acc: 0.338 - ETA: 7s - loss: 1.7576 - acc: 0.335 - ETA: 7s - loss: 1.7535 - acc: 0.336 - ETA: 7s - loss: 1.7563 - acc: 0.332 - ETA: 7s - loss: 1.7571 - acc: 0.333 - ETA: 6s - loss: 1.7538 - acc: 0.335 - ETA: 6s - loss: 1.7539 - acc: 0.334 - ETA: 6s - loss: 1.7532 - acc: 0.334 - ETA: 6s - loss: 1.7549 - acc: 0.331 - ETA: 6s - loss: 1.7581 - acc: 0.329 - ETA: 6s - loss: 1.7593 - acc: 0.329 - ETA: 6s - loss: 1.7594 - acc: 0.331 - ETA: 5s - loss: 1.7563 - acc: 0.333 - ETA

14341/14341 [==============================] - ETA: 8s - loss: 1.7017 - acc: 0.360 - ETA: 8s - loss: 1.7461 - acc: 0.330 - ETA: 8s - loss: 1.7116 - acc: 0.345 - ETA: 8s - loss: 1.6842 - acc: 0.363 - ETA: 8s - loss: 1.6758 - acc: 0.365 - ETA: 8s - loss: 1.6795 - acc: 0.365 - ETA: 8s - loss: 1.6750 - acc: 0.371 - ETA: 7s - loss: 1.6670 - acc: 0.374 - ETA: 7s - loss: 1.6664 - acc: 0.376 - ETA: 7s - loss: 1.6636 - acc: 0.380 - ETA: 7s - loss: 1.6704 - acc: 0.377 - ETA: 7s - loss: 1.6646 - acc: 0.380 - ETA: 7s - loss: 1.6587 - acc: 0.383 - ETA: 7s - loss: 1.6584 - acc: 0.385 - ETA: 7s - loss: 1.6561 - acc: 0.385 - ETA: 7s - loss: 1.6575 - acc: 0.382 - ETA: 6s - loss: 1.6589 - acc: 0.382 - ETA: 6s - loss: 1.6594 - acc: 0.381 - ETA: 6s - loss: 1.6650 - acc: 0.379 - ETA: 6s - loss: 1.6602 - acc: 0.382 - ETA: 6s - loss: 1.6543 - acc: 0.385 - ETA: 6s - loss: 1.6522 - acc: 0.386 - ETA: 6s - loss: 1.6513 - acc: 0.387 - ETA: 6s - loss: 1.6540 - acc: 0.386 - ETA: 5s - loss: 1.6533 - acc: 0.388 - ETA

14341/14341 [==============================] - ETA: 8s - loss: 1.6738 - acc: 0.335 - ETA: 8s - loss: 1.6098 - acc: 0.390 - ETA: 8s - loss: 1.5988 - acc: 0.408 - ETA: 8s - loss: 1.5785 - acc: 0.415 - ETA: 8s - loss: 1.5835 - acc: 0.420 - ETA: 8s - loss: 1.5853 - acc: 0.419 - ETA: 8s - loss: 1.5846 - acc: 0.422 - ETA: 8s - loss: 1.5951 - acc: 0.415 - ETA: 7s - loss: 1.5981 - acc: 0.412 - ETA: 7s - loss: 1.6038 - acc: 0.410 - ETA: 7s - loss: 1.5993 - acc: 0.416 - ETA: 7s - loss: 1.5998 - acc: 0.416 - ETA: 7s - loss: 1.5967 - acc: 0.416 - ETA: 7s - loss: 1.5912 - acc: 0.419 - ETA: 7s - loss: 1.5959 - acc: 0.417 - ETA: 7s - loss: 1.5973 - acc: 0.417 - ETA: 6s - loss: 1.5948 - acc: 0.415 - ETA: 6s - loss: 1.6018 - acc: 0.413 - ETA: 6s - loss: 1.6014 - acc: 0.412 - ETA: 6s - loss: 1.5936 - acc: 0.416 - ETA: 6s - loss: 1.5945 - acc: 0.414 - ETA: 6s - loss: 1.5936 - acc: 0.415 - ETA: 6s - loss: 1.5948 - acc: 0.413 - ETA: 6s - loss: 1.5934 - acc: 0.413 - ETA: 5s - loss: 1.5950 - acc: 0.411 - ETA

In [8]:
#이미지 샘플링 할 경우
# from keras.callbacks import EarlyStopping
# early_stopping = EarlyStopping()
# keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')

# from keras.preprocessing.image import ImageDataGenerator
# model = Sequential()
# model.add(Convolution2D(16, 3, 3, border_mode='same', activation='relu', #3x3필터 16개를 적용, same:입력이미지와 출력 이미지크기같음
#                         input_shape=data_train_image.shape[1:]))
# model.add(MaxPooling2D(pool_size=(2, 2))) #2,2 중 가장 높은값을 가져옴 - 사소한 특징 무시
# model.add(Dropout(0.25))
  
# model.add(Convolution2D(64, 3, 3,  activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
 
# model.add(Convolution2D(64, 3, 3))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
  
# model.add(Flatten())
# model.add(Dense(128, activation = 'relu')) # 256
# model.add(Dropout(0.5))
# model.add(Dense(8,activation = 'softmax'))
  
# model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

# #이미지 겹치기로 늘려주면 좋음
# datagen = ImageDataGenerator(
#         rotation_range=40,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         fill_mode='nearest') #try constant

# datagen.fit(data_train_image)

# model.fit_generator(datagen.flow(data_train_image,labels_train_image,batch_size =3),validation_data=(data_test_image, labels_test_image),verbose = 2, epochs=100,steps_per_epoch = len(data_test_image) / 32,callbacks=[early_stopping])

# model.save('Gersang.h5')

In [9]:
accr = model.evaluate(data_test_image,labels_test_image)

5976/5976 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 245us/step


In [10]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 1.685
  Accuracy: 0.368


In [11]:
# inception v3 trans learning
import tensorflow as tf
import keras.applications
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
base_model = InceptionV3(weights='imagenet', include_top=False)

CLASSES = 8
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
prev_model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model = Sequential()
model.add(prev_model)

model.add(Dense(256, activation = 'relu'))

model.add(Dense(8,activation = 'softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
from keras.applications.inception_v3 import preprocess_input
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
datagen.fit(data_train_image)

EPOCHS = 5
BATCH_SIZE = 32
STEPS_PER_EPOCH = 320
VALIDATION_STEPS = 64
MODEL_FILE = 'filename.model'
history = model.fit_generator(
    datagen.flow(data_train_image,labels_train_image,batch_size =3),
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=(data_test_image, labels_test_image),
    validation_steps=VALIDATION_STEPS)
  

# model.save(MODEL_FILE)

Epoch 1/5
320/320 [==============================] - ETA: 15:00 - loss: nan - acc: 0.333 - ETA: 2:08 - loss: nan - acc: 0.190 - ETA: 1:09 - loss: nan - acc: 0.17 - ETA: 44s - loss: nan - acc: 0.1500 - ETA: 33s - loss: nan - acc: 0.148 - ETA: 26s - loss: nan - acc: 0.166 - ETA: 21s - loss: nan - acc: 0.178 - ETA: 18s - loss: nan - acc: 0.170 - ETA: 16s - loss: nan - acc: 0.166 - ETA: 14s - loss: nan - acc: 0.163 - ETA: 12s - loss: nan - acc: 0.171 - ETA: 11s - loss: nan - acc: 0.164 - ETA: 10s - loss: nan - acc: 0.158 - ETA: 9s - loss: nan - acc: 0.161 - ETA: 8s - loss: nan - acc: 0.15 - ETA: 7s - loss: nan - acc: 0.16 - ETA: 7s - loss: nan - acc: 0.16 - ETA: 6s - loss: nan - acc: 0.16 - ETA: 6s - loss: nan - acc: 0.16 - ETA: 5s - loss: nan - acc: 0.16 - ETA: 5s - loss: nan - acc: 0.16 - ETA: 4s - loss: nan - acc: 0.16 - ETA: 4s - loss: nan - acc: 0.16 - ETA: 4s - loss: nan - acc: 0.16 - ETA: 3s - loss: nan - acc: 0.16 - ETA: 3s - loss: nan - acc: 0.16 - ETA: 3s - loss: nan - acc: 0.16 

320/320 [==============================] - ETA: 3s - loss: nan - acc: 0.0000e+ - ETA: 3s - loss: nan - acc: 0.2222   - ETA: 3s - loss: nan - acc: 0.27 - ETA: 2s - loss: nan - acc: 0.19 - ETA: 2s - loss: nan - acc: 0.17 - ETA: 2s - loss: nan - acc: 0.19 - ETA: 2s - loss: nan - acc: 0.19 - ETA: 2s - loss: nan - acc: 0.21 - ETA: 2s - loss: nan - acc: 0.21 - ETA: 2s - loss: nan - acc: 0.20 - ETA: 2s - loss: nan - acc: 0.19 - ETA: 2s - loss: nan - acc: 0.19 - ETA: 2s - loss: nan - acc: 0.18 - ETA: 1s - loss: nan - acc: 0.18 - ETA: 1s - loss: nan - acc: 0.18 - ETA: 1s - loss: nan - acc: 0.18 - ETA: 1s - loss: nan - acc: 0.18 - ETA: 1s - loss: nan - acc: 0.18 - ETA: 1s - loss: nan - acc: 0.18 - ETA: 1s - loss: nan - acc: 0.17 - ETA: 1s - loss: nan - acc: 0.16 - ETA: 1s - loss: nan - acc: 0.17 - ETA: 1s - loss: nan - acc: 0.17 - ETA: 1s - loss: nan - acc: 0.17 - ETA: 1s - loss: nan - acc: 0.18 - ETA: 1s - loss: nan - acc: 0.18 - ETA: 1s - loss: nan - acc: 0.17 - ETA: 1s - loss: nan - acc: 0.17